In [1]:
import pandas as pd
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Merge, Flatten

import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

Using Theano backend.


In [4]:
%%time
df = pd.read_pickle("data_2016_subset.pkl")
print(df.shape)
df.head()

(2000000, 14)
CPU times: user 10.7 s, sys: 5.3 s, total: 16 s
Wall time: 16.3 s


In [5]:
# Basic preprocessing
df = df[df.dest_country_codes.notnull() & df.stop.notnull()]
df.sort_values(['start','stop'],inplace=True)
df.drop_duplicates(subset=["start","impact_bps","impact_pps"],inplace=True)
# Filter out small bps attacks
df = df[df.impact_bps>32*df.impact_pps]
#filter out attacks bigger than 800Gbps
df = df[df.impact_gbps<1200]
df = df[~df.duration.isnull()]

In [6]:
df['triggers'] = [[trigger['name'] for trigger in triggers]
                  if len(triggers)>0 else None for triggers in df.triggers.values]
    
# get duration as float and log interfaces and routers
df["start_f"] = df.start.values.astype(float)/(1e9*3600*24)
df["duration_f"] = df.duration.values.astype(float)/(1e9*3600*24)
df['lg_interface'] = np.log10(1+df['infrastructure_num_interfaces'].values)
df['lg_routers'] = np.log10(1+df['infrastructure_num_routers'].values)
df['cc_idx'] = df.dest_country_codes.astype('category').cat.codes.values

In [7]:
df.head()

,dest_country_codes,direction_type,impact_bps,impact_pps,infrastructure_num_interfaces,infrastructure_num_routers,source_country_codes,start,stop,triggers,duration,impact_gbps,lg_impact_gbps,lg_impact_pps,start_f,duration_f,lg_interface,lg_routers,cc_idx
80079,BA,Incoming,80795088.0,9646.0,2.0,1.0,"[IN, ID, EG, BG, TR, MY, IL, TW, ES, NA, US, S...",2016-01-01 00:00:05,2016-01-01 00:05:28,[TCP SYN],00:05:23,0.080795,-1.092615,3.984347,16801.000058,0.003738,0.477121,0.30103,15
148,CO,Incoming,77679520.0,7297.0,0.0,0.0,[],2016-01-01 00:00:26,2016-01-01 00:40:56,[IP Fragmentation],00:40:30,0.077680,-1.109693,3.863144,16801.000301,0.028125,0.000000,0.00000,36
59961,KR,Outgoing,990187520.0,131583.0,16.0,1.0,[KR],2016-01-01 00:00:43,2016-01-01 00:08:11,[UDP],00:07:28,0.990188,-0.004283,5.119200,16801.000498,0.005185,1.230449,0.30103,96
40475,KE,Incoming,175704624.0,23007.0,0.0,0.0,[],2016-01-01 00:00:45,2016-01-01 00:04:59,[TCP SYN],00:04:14,0.175705,-0.755217,4.361860,16801.000521,0.002940,0.000000,0.00000,91
41167,CN,Incoming,410165664.0,54776.0,2.0,1.0,[DE],2016-01-01 00:00:45,2016-01-01 00:11:27,"[TCP SYN, Total Traffic]",00:10:42,0.410166,-0.387041,4.738590,16801.000521,0.007431,0.477121,0.30103,35


## Check correlation of variables

## Plot some histograms

## What are the most prominent Countries in attacks

## Fit the model to predict log impact gbps

In [2]:
# Todo: 
# Grab the x data (anything that is not categorical)
# Grab the categorical data
# grab the y variable

# scale the xs and ys to have mean 0 and stddev 1 (StandardScaler) 
#split all of above in training and testing (train_test_split)

In [14]:
def r2(y_true, y_pred):
    e = y_true - y_pred
    return 1 - K.mean(K.square(e),axis=-1)/K.var(y_true)

In [15]:
# Todo: complete the rest of the model

model.compile(loss="mse", optimizer='adagrad', metrics=[r2])

/root/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=5, units=32)`
/root/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
# Todo: test the model on test data